In [1]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    GIT_ROOT = 'https://github.com/furyhawk/text_summarization/raw/master'
    os.system(f'wget {GIT_ROOT}/notebooks/setup.py')

%run -i setup.py

You are working on a local system.
Files will be searched relative to "..".


In [3]:
%run "$BASE_DIR/settings.py"

%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'png'

# to print output of all statements and not just the last
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# otherwise text between $ signs will be interpreted as formula and printed in italic
pd.set_option('display.html.use_mathjax', False)

# path to import blueprints packages
sys.path.append(BASE_DIR + '/packages')

In [4]:
# adjust matplotlib resolution
matplotlib.rcParams.update({'figure.dpi': 200 })

In [5]:
from rouge_score import rouge_scorer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import tokenize
import matplotlib.pyplot as plt
import html
import re
import random
import rouge_score
import wikipediaapi

In [7]:
import requests
from bs4 import BeautifulSoup
import os.path
from dateutil import parser
import pandas as pd
import numpy as np

In [6]:
pd.options.display.max_colwidth = 150 ###
file = "un-general-debates-blueprint.csv"
file = f"{BASE_DIR}/data/un-general-debates/un-general-debates.csv.zip" ### real location
df = pd.read_csv(file)
df.sample(2, random_state=53)

session  year country  \
3871       42  1987     HND   
4697       35  1980     THA   

                                                                                                                                                       text  
3871  ﻿On behalf of the Government of Honduras I extend to you, Sir, sincere congratulations on your election as President of the forty-second session o...  
4697  ﻿My delegation takes pleasure in extending to Mr. Riidiger von Wechmar its sincere congratulations on his unanimous election to the presidency of ...

# Summarizing text using topic representation
## Identifying important words with TF-IDF values


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import tokenize
nltk.download('punkt')


sample_text = df['text'].iloc[1]
print("\nSample:", sample_text)
sentences = tokenize.sent_tokenize(sample_text)
tfidfVectorizer = TfidfVectorizer()
words_tfidf = tfidfVectorizer.fit_transform(sentences)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\furyx\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True


Sample: ﻿
May I begin by congratulating you. Sir, on your election to the presidency of the General Assembly at its forty-fourth session. Your wide experience in the United Nations makes you especially qualified to guide the Assembly and its deliberations. You have my best wishes for the demanding task that lies ahead of you. May I also express my thanks to the outgoing President, Mr. Dante Caputo, who so ably fulfilled his role on behalf of the Assembly. It gives me particular pleasure to pay tribute to the Secretary-General. We owe a great deal to his authority and leadership.
As we are assembled here for the annual general debate, every Member state needs to reflect on certain fundamental questions. Is the United Nations doing everything possible in order to meet its challenges? Are we making progress? Can we say that the United Nations enters the last decade of this century with determination and justified self-confidence? Is my own country doing everything it can in order to help

In [12]:
# Parameter to specify number of summary sentences required
num_summary_sentence = 3

# Sort the sentences in descending order by the sum of TF-IDF values
sent_sum = words_tfidf.sum(axis=1)
important_sent = np.argsort(sent_sum, axis=0)[::-1]

# Print three most important sentences in the order they appear in the article
for i in range(0, len(sentences)):
    if i in important_sent[:num_summary_sentence]:
        print (sentences[i])

Both industrialized and developing countries are directly affected by the depletion of the ozone layer through increasing emissions of carbon dioxides, which produce the so-called greenhouse effect in the climate of the Earth, and by the increasing presence of substances in the atmosphere that cause acid rain and direct dangers to health.
At the Conference the representatives of the more than 80 States present agreed unanimously to the Helsinki Declaration on the Protection of  the Ozone Layer, which contains an ambitious undertaking to phase out completely the use of ozone-depleting chlorofluorocarbons by the year 2000, or sooner if possible, and to phase out other ozone-depleting substances as soon as possible.
It is still too early to draw conclusions from the whole process of transition to independence, but we believe that the programme has been brought back on track thanks to the persistent efforts of the Secretary-General and the competent performance of the UNTAG personnel.


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import tokenize

def tfidf_summary(text, num_summary_sentence):
    summary_sentence = []
    sentences = tokenize.sent_tokenize(text)
    tfidfVectorizer = TfidfVectorizer()
    words_tfidf = tfidfVectorizer.fit_transform(sentences)
    sentence_sum = words_tfidf.sum(axis=1)
    important_sentences = np.argsort(sentence_sum, axis=0)[::-1]
    for i in range(0, len(sentences)):
        if i in important_sentences[:num_summary_sentence]:
            summary_sentence.append(sentences[i])
    return summary_sentence

In [16]:
where = df['country'].isin(['SGP'])
sgp_df = data=df[where]
sgp_df.head()

sample_text = df.query("year==2014 and country=='SGP'")['text'].values[0]
test_text = tfidf_summary(sample_text, 5)

print("\ntest text: ", test_text)

session  year country  \
64        44  1989     SGP   
160       25  1970     SGP   
240       68  2013     SGP   
449       40  1985     SGP   
661       63  2008     SGP   

                                                                                                                                                      text  
64   ﻿First of all, let me congratulate you on your election to the presidency of the General Assembly at its forty-fourth session. Your unique and ill...  
160   Mr. President, instead of offering you my congratulations as preceding speakers have done, I wish to thank you on behalf of my delegation. We tha...  
240  I warmly \ncongratulate Mr. John Ashe on his election as President \nat the sixty-eighth session of the General Assembly and \nwish him success du...  
449  Allow me to congratulate the President on his assumption of this high office. There are few men in this hall who have been associated with the Uni...  
661  The collapse of the recent \nDoha talks is deeply troubling. The Doha Development \nRound was the first round of talks to be launched after \nthe ...


test text:  ['Security \nCouncil resolution 2178 (2014), on foreign terrorist \nfighters, which Singapore co-sponsored, is an important \nstep to combat global terrorism and will be critical to \ncutting off financial and material support for ISIS and \npreventing the movement of foreign terrorist fighters.', 'Regarding the principles for sustainable development, \nSingapore is participating actively in the discussions \nfor the post-2015 period, focusing on those areas where \nwe can contribute — key principles that were critical \nto our development path and might be relevant to other \ndeveloping countries.', 'As noted on page \n17 of the report of the High-level Panel of Eminent \nPersons on the Post-2015 Development Agenda, cities \nare where the battle for sustainable development \nwill be won or lost.', 'This year, we \nwill observe that Day together with relevant United \nNations agencies and civil society organizations by \nfocusing on the serious problems faced by women and 